# Majority Judgment with 3-2-1 stability filtering

Majority Judgment is a median rating voting method.

Voters assign a score to each candidate.  Balinsky and Laracki recommend "named" scores (e.g. "Excellent", "Very Good", "Good", "Inadequate" "Poor", "Reject"), but numerically these are equivalent to letter grades of A through F, or numerical scores of 5 to 0.

The 3-2-1 stability filter requires an approval cutoff.  With 0-5 rating, scores of 5, 4, 3 (== A, B, C, respectively) are approved, and scores of 2, 1, 0 (== D, E, F) are disapproved.

Other scoring ranges typically used are: (a) 4-slot range of 0 to 3, with 3, 2, 1 approved, 0 disapproved; and (b) 0-9 range, with 9-5 approved and 4-0 disapproved.

The 3-2-1 filter is applied as follows:

* Semifinals:  Score the candidates using some measure that satisifes the Favorite Betrayal Criterion, and take the top 3 candidates
* Finals: drop the most disapproved to get the top 2 approved
* Winner: Pairwise winner between the finalists -- choose the candidate who is preferred over the other on the most ballots.

MJ321 chooses the semifinalists (the "3" stage) using the Majority Judgment grade, then follows 3-2-1.

The MJ median rating is found by taking the rating given by the middle voter.  If there is an even number of voters and the two middle voters give different ratings, the lower of the two ratings is used.

To break ties in median rating, one median score is removed at a time from each candidate's totals until one candidate gets a higher or lower rating.

For a given candidate, determine the totals at or above a given grade. Using the 0-5 rating, T0 is the total number of votes, T1 is the total number of votes at or above 1 (which equals T0 minus the number of 0 votes), T2 = the total number of votes at or above 2 (which equals T1 minus the number of 1 votes), etc.  

Next, determine the median rating by successively examining T5, T4, etc. until a total is found that exceeds 50% of the votes.

To determine the majority grade, sort the pairs {|50% - T_j|, j + sign(50% - T_j) if sign < 0} in ascending order by their absolute value.  This tells us what new median ratings would be found in succession by removing median votes in the case of a tie.  Then create a new list, {j_m, j0, T_j0, j1, T_j1, ...}.  This is the candidates's majority grade.

As an example, let's say a candidate has T5 = 10%, T4 = 30% (10% A grades + 20% B grades), T3 = 60%, T2 = 80%, and T1 = 89%.  T0 is always 100%.  The median rating is 3, because T4 is less than 50% and T3 is greater than 50%.  The sorted differences are

(|50% - 60%|, 2), (|50% - 30%|, 4), (|50% - 80%|, 1), (|50% - 89%|, 0), (|50% - 10%|, 5)

and the majority grade is

{3, 2, 60%, 4, 30%, 1, 80%, 0, 89%, 5, 10%}

Note that if there are K score levels, there are 2 k - 1 terms in the majority grade.


The majority grade ranking takes each candidate's majority grade and sorts in descending order.  That places the candidate with the highest j_m (median rating) at the top, and when two candidates are tied for the same median rating, the subsequent values in the majority grade are compared until a tie is broken.  For example, if the candidate above is compared with another candidate whose majority grade is {3, 2, 55%, ....}, the tie would be broken in favor of the first candidate, because the first candidate's 3rd term is bigger.

This might seem complicated, but the Tj totals are summable, and there are only K-1 totals to compute for each candidate.

First, set up for processing and plotting:

In [29]:
% matplotlib notebook
import matplotlib
% config InlineBackend.figure_format = 'pdf'
% matplotlib inline
# matplotlib.use('SVG')
import matplotlib.pyplot as plt
import numpy as np
from math import *

Next, let's test out making a 2D gaussian distribution with 4096 points, a sigma of 0.5 in each coordinate direction, and then plotting it:

In [32]:
from numpy.random import multivariate_normal
sample = multivariate_normal([0.,0.],[[0.25,0],[0,0.25]],10000)

sigsq = [0.25, 1, 2.25, 4]

tots = [0 for i in range(len(sigsq))]

for x, y in sample:
    radsq = x*x + y*y
    for i, s in enumerate(sigsq):
        if radsq <= s:
            tots[i] += 1

n = len(sample)

print([t/n for t in tots])


[0.3935, 0.8626, 0.9887, 0.9996]


In [33]:
plt.ion()

sp = plt.subplot(1,1,1, aspect=1)

plt.plot(sample[:,0],sample[:,1],'.')